Welcome to the world of Anime. This code requieres the dataset mentioned in Readme file of github . You can fork the notebook and can use to extract any other parts of the above mentioned site. Let's dive in.

The following cells of code will help you to extract the links to each anime character

In [ ]:
# importing the requirements
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen
import time
import os
import shutil

What I am doing web scraping. This notebook is also helpful for learning the web scraping.

In [ ]:
base_link1 = "https://myanimelist.net/character.php?letter="   # base path for every letter (A-Z)
base_link2 = "&show="             # base path for every sub pages of given letter (50 anime characters per page)
letters_ls = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
base_links = [base_link1 + i for i in letters_ls]

# these are the number of pages per each alphabet. 
#For exmaple the anime characters with starting letter "Z" are present in 7 pages 
#(50 per each page and last page may not contain 50 images)
num_dict = {}
num_dict["Z"] = 7
num_dict["Y"] = 67
num_dict["X"] = 2
num_dict["W"] = 22
num_dict["V"] = 13
num_dict["U"] = 23
num_dict["T"] = 122
num_dict["S"] = 186
num_dict["R"] = 29
num_dict["Q"] = 2
num_dict["P"] = 12
num_dict["O"] = 62
num_dict["N"] = 74
num_dict["M"] = 146
num_dict["L"] = 26
num_dict["K"] = 206
num_dict["J"] = 16
num_dict["I"] = 61
num_dict["H"] = 115
num_dict["G"] = 29
num_dict["F"] = 41
num_dict["E"] = 20
num_dict["D"] = 26
num_dict["C"] = 31
num_dict["B"] = 32
num_dict["A"] = 103

print(sum(list(num_dict.values())))

This block will give you links to each page of each letter (ie., 7 links for "Z" letter)

In [ ]:
final_links = []

for num,letter in enumerate(letters_ls):
    for i in range(num_dict[letter]):
        if i==0:
            final_links.append(base_links[num])
        else:
            final_links.append(base_links[num]+base_link2+str(i*50))
len(final_links)

On running below block you will get final links ie., each link contains details of one anime character. Basically I used 

> time.sleep()

to  make sure that there is no much traffic from our side during the web scraping. As we are going through a lot of pages. it is better to use the sleep to make sure that we do not get the [Error 403](https://stackoverflow.com/questions/16627227/http-error-403-in-python-3-web-scraping/31758803).

In [ ]:
a_tags = []
time.sleep(180)
for i,link in enumerate(final_links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'}
    req = Request(link , headers=headers)
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "html.parser")
    links = soup.findAll("a")
    for l in links:
        ls = []
        if l.img:
            a_tags.append(l)
    if i%150==0 and i!=0:
        print("Completed {} links and waiting for 300 seconds".format(i))
        time.sleep(300)
        print("Waiting completed, going for next set of 150 links")

In [ ]:
# This code gives you the actual links form a tag that we got from previous csv file
anime_links = []
for link in a_tags:
    if "onclick" in str(link):
        a_tags.remove(link)
    else:
        anime_links.append(link.attrs["href"])

In [ ]:
# Creating a new folder where we store the images

if os.path.isdir("./dataset/") == False:
    os.mkdir("./dataset/")

In [ ]:
df_links = pd.read_csv("../input/anime-names-and-image-generation/anime_links.csv")
anime_links = list(df_links["Link"])
len(anime_links)     #72992

Following block gives you final images stored in dataset folder (./dataset). You can also extract names of characters form name of each anime image.

In [ ]:

anime_names = []

for i,src in enumerate(anime_links):
    src_new = src.encode('ascii', 'ignore').decode('ascii')       #there are some names where there are characters other than ascii. To reduce complexity iam excluding those
    if src_new != src:
        continue
    req = Request(src , headers=headers)
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "html.parser")
    links = soup.findAll("a")
    for link in links:
        if link.img and "border" not in str(link) and "onclick" not in str(link):  # actual image links
            split= str(link.img).split("\"")
            img_src = split[5]
            name = split[1]
            
            if img_src.split(".")[-1] == "jpg" and len(name.split("/"))==1:
                split = str(link.img).split("\"")
                response = requests.get(img_src, headers=headers)
                image_name = "./dataset/"+name + ".jpg"
                file = open(image_name, "wb")
                file.write(response.content)
                file.close()

    if i%120==0 and i!=0:  # downloading 120 images and waiting for 4 min and then the loop continues.
        print("Completed {} links and waiting for 240 seconds".format(i))
        time.sleep(240)
        print("Waiting completed, going for next set of 120 links\n\n")